In [ ]:
import pandas
import pulp

s_dfm = pandas.read_csv('students.csv')
s_pair_dfm = pandas.read_csv('student_pairs.csv')

prob = pulp.LpProblem('ClassAssignmentProblem', pulp.LpMaximize)

# 生徒のリスト
S = s_dfm.student_id.tolist()

# クラスのリスト
C = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

# 生徒とクラスのペアのリスト
SC = [(s,c) for s in S for c in C]

# 生徒をどのクラスに割り当てるを変数として定義
x = pulp.LpVariable.dicts('x', SC, cat='Binary')

# (1)各生徒は１つのクラスに割り当てる
for s in S:
    prob += pulp.lpSum([x[s,c] for c in C]) == 1

# (2)各クラスの生徒の人数は39人以上、40人以下とする。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S]) >= 39
    prob += pulp.lpSum([x[s,c] for s in S]) <= 40

# 男子生徒のリスト
S_male = [row.student_id for row in s_dfm.itertuples() if row.gender == 1]

# 女子生徒のリスト
S_female = [row.student_id for row in s_dfm.itertuples() if row.gender == 0]

# (3) 各クラスの男子生徒、女子生徒の人数は20人以下とする。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_male]) <= 20
    prob += pulp.lpSum([x[s,c] for s in S_female]) <= 20

# 学力を辞書表現に変換
score = dict([(row.student_id, row.score) for row in s_dfm.itertuples()])

# 平均点の算出
score_mean = s_dfm.score.mean()

# (4) 各クラスの学力試験の平均点は学年平均点±10点とする。
for c in C:
    prob += pulp.lpSum([x[s,c]*score[s] for s in S]) >= (score_mean - 10) * pulp.lpSum([x[s,c] for s in S])
    prob += pulp.lpSum([x[s,c]*score[s] for s in S]) <= (score_mean + 10) * pulp.lpSum([x[s,c] for s in S])

# リーダー気質の生徒の集合
S_leader = [row.student_id for row in s_dfm.itertuples() if row.leader_flag == 1]

# (5)各クラスにリーダー気質の生徒を2人以上割り当てる。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_leader]) >= 2

# 特別な支援が必要な生徒の集合
S_support = [row.student_id for row in s_dfm.itertuples() if row.support_flag == 1]

# (6) 特別な支援が必要な生徒は各クラスに1人以下とする。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_support]) <= 1

    # 生徒の特定ペアリスト
SS = [(row.student_id1, row.student_id2) for row in s_pair_dfm.itertuples()]

# (7) 特定ペアの生徒は同一クラスに割り当てない。
for row in s_pair_dfm.itertuples():
    s1 = row.student_id1
    s2 = row.student_id2
    for c in C:
        prob += x[s1,c] + x[s2,c] <= 1

#ここから修正する        
        
        
# 初期クラス編成のデータを作成
# 学力をもとに順位を付与
s_dfm['score_rank'] = s_dfm.score.rank(ascending=False, method='first') 


# 学力順にクラス編成し、init_assigned_classカラムを作成
class_dic = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H'}
s_dfm['init_assigned_class'] = s_dfm.score_rank.map(lambda x:x % 8).map(class_dic)
s_dfm.head()        
        
    
# init_flagを作成
init_flag = dict([((s,c), 0)for s in S for c in C])
for row in s_dfm.itertuples():
    init_flag[row.student_id, row.init_assigned_class] = 1
    
    
# 目的関数:初期クラス編成と最適化結果のクラス編成をできるだけ一致させる
prob.objective = pulp.lpSum([x[s,c] * init_flag[s,c] for s,c in SC])        


# 求解
status = prob.solve()
print('Status:', pulp.LpStatus[status])

# 最適化結果の表示
# 各クラスに割り当てられている生徒のリストを辞書に格納
C2Ss = {}
for c in C:
    C2Ss[c] = [s for s in S if x[s,c].value()==1]

for c, Ss in C2Ss.items():
    print('Class:', c)
    print('Num:', len(Ss))
    print('Student:', Ss)
    print()

In [ ]:
#一応確認
result_dfm2 = s_dfm.copy()

S2C = {}
for s in S:
    S2C[s] = [c for c in C if x[s,c].value()==1][0]
          
result_dfm2['assigned_class'] = result_dfm2.student_id.map(S2C)
result_dfm2.head(5)


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,20))
for i, c in enumerate(C):
    cls_dfm = result_dfm2[result_dfm2.assigned_class==c]
    ax = fig.add_subplot(4, 2, i+1)
    ax.hist(cls_dfm.score, bins=range(0,500,40))
    ax.set_title('Class:{:s}'.format(c))
    ax.set_xlim(0, 500)
    ax.set_ylim(0,15)
    ax.set_xlabel('score')
    ax.set_ylabel('num')